## NLP Preprocessing of CUDA dataset


In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
os.getcwd()

'/Users/gustavhartz/Projects/School/legal-contract-elements/notebooks'

We create a dataframe as in the drqa dataset. The filters applied will be to filter cases where there doesn't exists a signle answer or where the dataset has split the answer into multiple sections 

In [152]:
import json
data = []
i=0
with open("../data/raw/CUAD_v1/CUAD_v1.json", encoding="utf-8") as f:
    cuad = json.load(f)
    # Contract
    for example in cuad["data"]:
        title = example.get("title", "").strip()
        # Paragraph in contract
        # We only look at the first one
        for paragraph in example["paragraphs"]:
            context = paragraph["context"].strip()
            for qa in paragraph["qas"]:
                if qa.get("is_impossible"):
                    continue
                question = qa["question"].strip()
                id_ = qa["id"]

                answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                answers = [answer["text"].strip() for answer in qa["answers"]]
                answer_end = [context.find(x)+len(x) for x in answers if context.find(x)]
                if len(answers)>1:
                    continue
                if answer_end and len(context[min(answer_starts):max(answer_end)])<2000:
                    data.append((id_, question, answers[0], answer_starts[0], answer_end[0], context[max(1,min(answer_starts)-200):min(len(context),max(answer_end)+500)]))

In [153]:
df =pd.DataFrame(data, columns=["id_", "question", "answer", "answer_start", "answer_end", "context"])

In [154]:
df.question.unique()

array(['Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
       'Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract',
       'Highlight the parts (if any) of this contract related to "Expiration Date" that should be reviewed by a lawyer. Details: On what date will the contract\'s initial term expire?',
       'Highlight the parts (if any) of this contract related to "Renewal Term" that should be reviewed by a lawyer. Details: What is the renewal term after the initial term expires? This includes automatic extensions and unilateral extensions with prior notice.',
       'Highlight the parts (if any) of this contract related to "Governing Law" that should be reviewed by a lawyer. Details: Which state/country\'s law governs the interpretation of the contract?',
       'Highlight the parts (if any) of 

In [162]:
df = df[df['question'].isin(['Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
                    'Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract',
                    """'Highlight the parts (if any) of this contract related to "Expiration Date" that should be reviewed by a lawyer. Details: On what date will the contract\'s initial term expire?'""",
                    'Highlight the parts (if any) of this contract related to "Renewal Term" that should be reviewed by a lawyer. Details: What is the renewal term after the initial term expires? This includes automatic extensions and unilateral extensions with prior notice.',
                    ])]

In [163]:
df.head(5)

,id_,question,answer,answer_start,answer_end,context
0,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,Highlight the parts (if any) of this contract ...,DISTRIBUTOR AGREEMENT,44,65,XHIBIT 10.6\n\n D...
1,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,Highlight the parts (if any) of this contract ...,"7th day of September, 1999.",263,290,NT\n\n THIS DISTRIBUTOR AGREEMENT (t...
3,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,Highlight the parts (if any) of this contract ...,If Distributor comp...,5543,5881,"ears (the ""Term"") which shall commence on th..."
8,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",Highlight the parts (if any) of this contract ...,Promotion and Distribution Agreement,307,343,"CH PORTION, WHICH HAS BEEN OMITTED HEREIN AND..."
14,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,Highlight the parts (if any) of this contract ...,SUPPLY CONTRACT,14,29,xhibit 10.16 SUPPLY CONTRACT Contract No: Date...


In [174]:
def normalize_spaces(text, remove_extra_spacing=False):
    '''
    Removes extra white spaces from the context.
    '''
    text = re.sub(r'\s', ' ', text)
    text = re.sub(r'\n', ' ', text)
    if remove_extra_spacing:
        text = re.sub(r' +', ' ', text)
    return text

In [175]:
df.context.apply(normalize_spaces, args=(True,))

0       XHIBIT 10.6 DISTRIBUTOR AGREEMENT THIS DISTRIB...
1       NT THIS DISTRIBUTOR AGREEMENT (the "Agreement"...
3       ears (the "Term") which shall commence on the ...
8       CH PORTION, WHICH HAS BEEN OMITTED HEREIN AND ...
14      xhibit 10.16 SUPPLY CONTRACT Contract No: Date...
                              ...                        
4017     executed in any number of counterparts all of...
4018    separately with the Secretary of the Commissio...
4019    xhibit 10.1 PROMOTION AGREEMENT by and between...
4022     Section 8.1 Term. The term of this Agreement ...
4027    . Counterparts 11 Page 2 of 12 Source: PERFORM...
Name: context, Length: 1060, dtype: object